In [5]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import enchant
from re import compile as _Re
import re
from nltk import ngrams
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
n = 3
MAX_VOCAB = 9999

In [3]:
def tfidf(df):
    docs = []
    #make docs
    for row in df.values:
        doc = []
        for idx, inst in enumerate(row[2:6]):
            if inst != inst:
                inst = '<NAN>'
            elif idx == 1 or idx == 2:                                       #title, body_html
                string = inst.replace('\n', '')
                string = re.sub(r'[a-zA-z+]', r'', string)
                string = re.sub(r'[^\w\s]', r'', string)
                string = re.sub(r'[0-9]+', r'', string)
                string = string.replace(' ', '')
                string = re.sub(r'(.)', '\g<1> ', string)
                doc.append(string)
            else:
                doc.append(inst)
            sent = ' '.join(doc)
        docs.append(sent)
        del sent, doc, string
        
    #form tfidf table    
    cv = CountVectorizer(tokenizer = lambda x: x.split(), ngram_range = (1, n))
    word_count_vector = cv.fit_transform(docs)
    tfidf_trans = TfidfTransformer(smooth_idf = True,use_idf = True) 
    tfidf_trans.fit(word_count_vector)
    wordcount = cv.transform(docs)
    tfidf_vectors = tfidf_trans.transform(wordcount)
    del cv, word_count_vector, tfidf_trans, wordcount
    
    #form id to tfidf dictionary
#    tfidf_vectors = dict([(identity, tfidf_vectors[idx].toarray().squeeze()) for idx, identity in enumerate(df['id'])])
#     id_to_tfidf = {}
#     for idx, identity in enumerate(df['id']):
#         id_to_tfidf[identity] = tfidf_vectors[idx].toarray().squeeze()
#     del tfidf_vectors
    return tfidf_vectors

In [4]:
def temp_mapping(t):                                        #map temperature categories to words
    mapped = []
    for temp_type in t:
        if temp_type == 1:
            mapped.append('room temperature')
        elif temp_type == 2:
            mapped.append('cold')
        else:
            mapped.append('frozen')
    return mapped
def get_emb_weight(path):                                        #form the glove file into vector
    emb_dict = {}
    with open(path, 'r', encoding='utf-8') as emb_file:
        for line in emb_file:
            vec = line.split()
            word = vec[0]
            emb_dict[word] = np.array((vec[1:]), dtype = float)
    return emb_dict
if __name__ == '__main__':
    data_name = 'undup_seo'
    save_path = './embedded/'
    
    #read in raw data
    print('Reading raw data...')
    with open('{}.pickle'.format(data_name), 'rb') as file:
        df = pickle.load(file)
    df = df.drop(df[df['price'] == 0].index)
    df = df.drop(df[df['price'] > 1000000].index)
    
    #get tfidf dictionary for product
    print('Forming tfidf vectors...')
    tfidf_vectors = tfidf(df)
    print('Saving tfidf vectors...')
    with open('./idx_to_tfidf.pickle', 'wb') as out:
        pickle.dump(tfidf_vectors, out)
    
    #append price info to product feature vector
    print('Concatenating price information...')
    for idx, inst in enumerate(df['price']):
        if inst > 1.7976931348623157e+308:
            df['price'][idx] = 1
    tfidf_vectors = np.append(tfidf_vectors.toarray(), np.array(df['price'])[:, np.newaxis], axis = 1)
    

Reading raw data...
Forming tfidf vectors...


KeyboardInterrupt: 

In [ ]:
wcss = []
for i in range(1, 20):
    kmeans = KMeans(n_clusters = i, max_iter = 300, n_init = 10, init = 'k-means++')
    kmeans.fit(tfidf_vectors)
    wcss.append(kmeans.inertia_)
    print()
plt.plot(range(1, 11), wcss)
plt.title('The Elbow Method')
plt.ylabel('WCSS')
plt.show()

# Fitting K-Means to the dataset
kmeans = KMeans(n_clusters = 5, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(X)

In [12]:
# chinese_splitter = _Re( '(?s)((?:[\ud800-\udbff][\udc00-\udfff])|.)' ).split
# d = enchant.Dict("en_US")
# doc = []
# for row in df.values:
#     r = ''
#     if(row[1] == row[1]):
#         fixed_str = fix_str(row[1])
#         split_str = fixed_str.split()    # seperate chinese and english
#         for text in split_str:
#             if d.check(text):            #is english
#                 r += text
#                 r += ' '
#             else:
#                 for c in chinese_splitter(text):
#                     r += c
#                     r += ' '
            
#     doc.append(r)

KeyboardInterrupt: 

In [ ]:
#     data_name = 'big_query_results'
#     df = pd.read_csv('./{}.csv'.format(data_name))
#     save_path = './embedded/'
    
#     title = df['title']
#     body_html = df['body_html']
#     price = df['price']
#     temperature_type = df['temperature_type']
#     t_type = temp_mapping(temperature_type)
    
#     sampled = {'title' : title, 'body_html' : body_html, 'price' : price, 'temp' : t_type}
#     sampled = pd.DataFrame(sampled)
#     train, test = train_test_split(sampled, test_size = 0.1, random_state = 0)
#     with open(save_path + 'sampled_train.pickle', 'wb') as out:
#         pickle.dump(train, out)
#     with open(save_path + 'sampled_test.pickle', 'wb') as out:
#         pickle.dump(test, out)

# def ngrams_with_tfidf(df):
#     docs = []
#     tf = defaultdict(int)
#     for idx, row in enumerate(df.values):
#         string = re.sub(r'(.)', '\g<1> ', row[4])            #pick out body_html
#         docs.append(string)
#     corpus = ' '.join(docs)
#     n_gram = ngrams(corpus, n)
#     term_freq = collections.Counter(n_gram)
#     term_freq = term_freq.most_common(WORD_FREQ)
    
#     #tf calculation
#     for combi in term_freq:                
#         w_pair, w_freq = combi[0], combi[1]
#         tf[w_pair] = w_freq
        
#     #idf calculation
#     idf = defaultdict(int)
#     for sent in docs:
#         seen = set()
#         words = sent.split()
#         for idx in range(len(words) - n + 1):
#             word_combi = tuple(words[idx : idx + n])
#             if word_combi not in seen:
#                 seen.union(word_combi)
#                 idf[word_combi] += 1
    
#     #form